In [ ]:
import os
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

# CUDA 디바이스 0, 2만 사용하도록 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from dataclasses import dataclass

import torch
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
    PreTrainedTokenizerBase
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')


@dataclass
class Config:
    #밑에 저장 이름도
    output_dir: str = 'stage2_attpool_cls_2l'
    checkpoint: str = "team-lucid/deberta-v3-base-korean"  # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 512
    n_splits: int = 20
    fold_idx: int = 0
    optim_type: str = "adamw_torch"
    per_device_train_batch_size: int = 16
    gradient_accumulation_steps: int = 1  # global batch size is 16
    per_device_eval_batch_size: int = 8
    n_epochs: int = 5
    freeze_layers: int = 0  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 1e-5
    warmup_steps: int = 20
    lora_r: int = 64
    lora_alpha: float = 16
    lora_dropout: float = 0.
    lora_bias: str = "none"

config = Config()

training_args = TrainingArguments(
    output_dir=config.output_dir,
    do_eval=True,
    overwrite_output_dir=True,
    report_to="wandb",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    # save_steps=200,
    optim=config.optim_type,
    weight_decay=1e-2,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
    dataloader_num_workers=64
)

tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.truncation_side='right'
tokenizer.add_eos_token = True


###
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoConfig, AutoModel, PreTrainedModel


class AttentionPooling(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionPooling, self).__init__()
        self.attention = nn.Linear(hidden_size, 1)

    def forward(self, last_hidden_state, attention_mask):
        scores = self.attention(last_hidden_state).squeeze(-1)
        scores = scores.masked_fill(attention_mask == 0, float('-inf')) #B x L
        attn_weights = F.softmax(scores, dim=1).unsqueeze(-1) #B x L x 1
        
        weighted_sum = torch.sum(last_hidden_state * attn_weights, dim=1)
        return weighted_sum


class AiModel(PreTrainedModel):
    """
    Hugging Face Trainer-compatible AI Model with Attention Pooling
    """
    def __init__(self, config):
        super().__init__(config)
        print("Initializing the AI Model...")

        self.backbone = AutoModel.from_pretrained(config._name_or_path, config=config)
        hidden_size = self.backbone.config.hidden_size

        self.pool = AttentionPooling(hidden_size)

        # self.projection_head = nn.Linear(hidden_size, 1)
        # self.projection_head = nn.Linear(hidden_size, 2) #1 -> 2
        # self.projection_head = nn.Linear(hidden_size*2, 2) #1 -> 2 (use cls concat)

        self.projection_head = nn.Sequential(nn.Linear(hidden_size*2, hidden_size, bias = True),
                                        nn.Linear(hidden_size, 2, bias = True),
                                        nn.Dropout(p=0.1, inplace=False)) #1 (use cls concat)
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=False,
        )

        ###BASE
        # last_hidden_state = outputs.last_hidden_state #B x 508 x H(768)
        # pooled_output = self.pool(last_hidden_state, attention_mask) #B x H(768)
        # logits = self.projection_head(pooled_output).squeeze(-1)  # shape: (batch_size)
        # logits = self.projection_head(pooled_output).squeeze(-1)  # shape: (batch_size)


        # ## USE CLS concat
        cls_hidden = outputs.last_hidden_state[:, 0, :] #B x 1 x H
        rest_hidden_state = outputs.last_hidden_state[:, 1:, :]  #B x (508-1) x H(768)
        pooled_output = self.pool(rest_hidden_state, attention_mask[:, 1:]) #B x H(768)

        my_hidden = torch.concat([cls_hidden, pooled_output], dim = 1) #B x H*2
        logits = self.projection_head(my_hidden)  # shape: B x 2
        # ##-----------

        ##-----------

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels.float())

        return {
            "loss": loss,
            "logits": logits,
        }

###-------------------------------------
from transformers import AutoConfig, AutoModel, PreTrainedModel

m_config = AutoConfig.from_pretrained(config.checkpoint)
model = AiModel(m_config)

model.config.use_cache = False

from datasets import load_from_disk

ds = load_from_disk("./stage1_llm")
if False:
    ds = ds.select(range(200))
    # 첫 번째 샘플의 labels만 1로 수정
ds = ds.map(lambda x: {'labels': [x['labels'], x['generated']]})

# Step 1: 'generated' 컬럼 제거
ds = ds.remove_columns(['generated'])

# Step 2: 컬럼 이름 변경
ds = ds.rename_columns({
    "original_input_ids": "input_ids",
    "original_attention_mask": "attention_mask",
    "original_token_type_ids": "token_type_ids",
})


#
#---------

from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import torch
from transformers import EvalPrediction

def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # shape: (batch_size, 2)
    labels = eval_preds.label_ids   # shape: (batch_size, 2)

    probs = torch.from_numpy(preds).float().sigmoid().numpy()  # shape: (batch_size, 2)
    binary_preds = (probs >= 0.5).astype(int)

    results = {}
    task_names = ["labels", "generated"]

    for i, task in enumerate(task_names):
        y_true = labels[:, i]
        y_prob = probs[:, i]
        y_pred = binary_preds[:, i]

        results[f"{task}_log_loss"] = log_loss(y_true, y_prob)
        results[f"{task}_auc"] = roc_auc_score(y_true, y_prob)
        results[f"{task}_accuracy"] = accuracy_score(y_true, y_pred)
        break
    return results

from math import ceil

def get_train_val_split_indices(dataset, n_splits):
    total = len(dataset)
    split_size = ceil(total / n_splits)
    indices = list(range(total))

    # Split into n_splits parts
    splits = [indices[i*split_size:(i+1)*split_size] for i in range(n_splits)]

    # Train: all but last split
    train_indices = [i for split in splits[:-1] for i in split]
    val_indices = splits[-1]

    return train_indices, val_indices

# 예시 사용:
train_idx, eval_idx = get_train_val_split_indices(ds, config.n_splits)

from transformers import Trainer
import torch.nn as nn

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")  # shape: (batch_size, 2)
        outputs = model(**inputs)
        logits = outputs.get("logits")  # shape: (batch_size, 2)

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss


trainer = CustomTrainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

ds = Dataset.from_csv("./test.csv")

class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch: dict) -> dict:
        title = ["<Title>: " + t for t in batch["title"]]
        para = ["\n\n<Full text>: " + t for t in batch["paragraph_text"]]
        texts = [t + p for t, p in zip(title, para)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)

        return {**tokenized}
    
encode = CustomTokenizer(tokenizer, max_length=512)
ds = ds.map(encode, batched=True)

from tqdm import tqdm
from transformers.data.data_collator import pad_without_fast_tokenizer_warning


@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(ds, model, batch_size=1):
    preds = []
    pseudo = []
    model.eval()
    
    for start_idx in tqdm(range(0, len(ds), batch_size)):
        end_idx = min(start_idx + batch_size, len(ds))
        tmp = ds[start_idx:end_idx]
        input_ids = tmp["input_ids"]
        attention_mask = tmp["attention_mask"]
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to("cuda:0"))
        proba = outputs.get("logits").cpu()
        
        preds.extend(proba[:, 0].tolist())
        pseudo.extend(proba[:,1].tolist())
    
    return preds, pseudo

a, b = inference(ds, model)

import pandas as pd


sub = pd.read_csv('./sample_submission.csv')
sub.head()

sub['generated'] = b

sub.to_csv('stage2_attpool_b.csv', index=False)